# Walk Combine

Iterate through gps-walks folder, limiting to one point every 5 seconds, converting to GPX + CSV format.

Copyright 2024 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import sys
import csv

from lxml import etree

import traceback

## Main Function

In [2]:
def loadCoordinates(path, name):
    '''Load coordinates from CSV file'''

    coordinates = []
    
    csvPath = os.path.join(path, name)
    with open(csvPath, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)

        for values in csvReader:
            coordinate = '{},{}'.format(values['lon'], values['lat'])
            coordinates.append(coordinate)

    return coordinates


def processFile(path, name, element):
    '''Process CSV file, converting to a LineString'''

    # Placemark
    placemark = etree.SubElement(element, 'Placemark')
    placemarkName = etree.SubElement(placemark, 'name')
    placemarkName.text = os.path.splitext(name)[0]
    placemarkStyleUrl = etree.SubElement(placemark, 'styleUrl')
    placemarkStyleUrl.text = '#wizard'

    # LineString
    lineString = etree.SubElement(placemark, 'LineString')
    lineStringExtrude = etree.SubElement(lineString, 'extrude')
    lineStringExtrude.text = '1'       
    lineStringTessellate = etree.SubElement(lineString, 'tessellate')
    lineStringTessellate.text = '1'

    # Coordinates
    coordinates = etree.SubElement(lineString, 'coordinates')
    coordinatesList = loadCoordinates(path, name)
    coordinates.text = '\n'.join(coordinatesList)


def processFolders(name, tree, element):
    '''Process folders'''

    # Folder
    folder = etree.SubElement(element, 'Folder')
    folderName = etree.SubElement(folder, 'name')
    folderName.text = name

    # Only open the folder if there are sub-folders
    if tree['dirs']:
        folderOpen = etree.SubElement(folder, 'open')
        folderOpen.text = '1'

    # Sub-folders
    for dir in sorted(tree['dirs'].keys()):
        processFolders(dir, tree['dirs'][dir], folder)

    # Files
    for file in sorted(tree['files']):
        processFile(tree['path'], file, folder)


def prepareKml(tree, name='GPS Walks'):
    '''Prepare KML prior to being saved'''

    # XML document
    xsi = 'http://www.w3.org/2001/XMLSchema-instance'
    kml = etree.Element(
        'kml', 
         {etree.QName(xsi, 'schemaLocation'): 'http://www.opengis.net/kml/2.2 http://schemas.opengis.net/kml/2.2.0/ogckml22.xsd'},
         nsmap={None: 'http://www.opengis.net/kml/2.2', 'xsi': xsi})

    # Document
    document = etree.SubElement(kml, 'Document')
    documentName = etree.SubElement(document, 'name')
    documentName.text = name
    documentOpen = etree.SubElement(document, 'open')
    documentOpen.text = '1'

    # Style
    style = etree.SubElement(document, 'Style')
    style.attrib['id'] = 'wizard'

    # LineStyle
    linestyle = etree.SubElement(style, 'LineStyle')
    linestyleColor = etree.SubElement(linestyle, 'color')
    linestyleColor.text = 'a00000ff'
    linestyleWidth = etree.SubElement(linestyle, 'width')
    linestyleWidth.text = '2'       

    # Process all of the folders
    processFolders('All walks', tree, document)

    return kml


def formatKml(kml):
    '''Format KML'''
    
    return etree.tostring(kml, pretty_print=True, xml_declaration=False, encoding='UTF-8',
                             doctype='<?xml version="1.0" encoding="UTF-8"?>')


def saveKml(kml):
    '''Save KML to file'''

    dataDir = os.path.normpath(os.path.join(projdir, '..', 'gps-walks', 'data'))
    KmlDir = os.path.join(dataDir, 'kml')

    filename = os.path.join(KmlDir, 'walks.kml')
    with open(filename, 'wb') as f:
        buffer = formatKml(kml)
        f.write(buffer)

In [3]:
def combineWalks():
    '''Iterate through CSV folders, combining into a single KML file'''

    dataDir = os.path.normpath(os.path.join(projdir, '..', 'gps-walks', 'data'))
    csvDir = os.path.join(dataDir, 'csv')

    tree = {'path': csvDir, 'dirs': {}, 'files': []}

    for root, dirs, files in os.walk(csvDir):
        node = tree
        rootParts = root.replace(csvDir, '').split(os.path.sep)[1:]
        for rootPart in rootParts:
            node = node['dirs'][rootPart]
    
        for dir in dirs:
            emptyNode = {'path': os.path.join(root, dir), 'dirs': {}, 'files': []}
            node['dirs'][dir] = emptyNode

        for file in files:
            ext = os.path.splitext(file)[1].lower()
            if ext and ext in ['.csv']:
                node['files'].append(file)

    kml = prepareKml(tree)

    saveKml(kml)

In [4]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    combineWalks()
    
    print(os.linesep + 'All done!')


All done!
